In [1]:
import cv2
import numpy as np

def cv_show(img,name="image"):
    cv2.imshow(name,img)
    cv2.waitKey() #ms 不输入代表任意键销毁窗口
    cv2.destroyAllWindows()

In [4]:
img1 = cv2.imread('01_Picture/19_Box.png',0) # 一本书
img2 = cv2.imread('01_Picture/20_Box_in_scene.png',0) # 多本书
cv_show(img1)
cv_show(img2)

In [6]:
sift = cv2.xfeatures2d.SIFT_create()
kp1, dst1 = sift.detectAndCompute(img1, None)
kp2, dst2 = sift.detectAndCompute(img2, None)

# crossCheck 表示两个特征点要互相匹配，例如 A 中的第 i 个特征点与 B 中第 j 个特征点最近的，并且 B 中第 j 个特征点到 A 中的第 i 个特征点也是最近的。      
# 将两幅图像的特征点、特征向量算出来，用欧氏距离去比较特征向量相似性，一般情况下默认用的是归一化后的欧式距离去做，为了使得结果更均衡些。
# 如果不用 sift 特征计算方法去做，而是用其他特征计算方法需要考虑不同的匹配方式。
bf = cv2.BFMatcher(crossCheck = True)  # cv2.BFMatcher 蛮力匹配缩写
print(dst1.shape, dst2.shape)

(603, 128) (969, 128)


## 一对一的匹配

In [17]:
matches = bf.match(dst1,dst2)
print("shape:",np.array(matches).shape) # 匹配得到的一对一的结果
print(matches[:3])
list(map(lambda x: x.distance, matches))[:5]

shape: (392,)
[<DMatch 0000019A063EC150>, <DMatch 0000019A063EC070>, <DMatch 0000019A063EC530>]


[212.0023651123047,
 244.86526489257812,
 119.62859344482422,
 341.0909118652344,
 296.07598876953125]

In [21]:
# 把匹配得到的结果从小到大排序
matches = sorted(matches, key = lambda x: x.distance)
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:3], outImg=None, flags=2)
cv_show(img3)

# K对最佳匹配

In [64]:
bf = cv2.BFMatcher()

sift = cv2.xfeatures2d.SIFT_create()
kp1, dst1 = sift.detectAndCompute(img1, None)
kp2, dst2 = sift.detectAndCompute(img2, None)

matches = bf.knnMatch(dst1, dst2, k=2) # k=2 意味着 1v2 匹配

print(matches[0]) # 内有两个元素 [<DMatch 0000019A063EC3F0>, <DMatch 0000019A04AF65D0>]

# queryIdx: 待匹配图片特征索引, trainIdx: 被匹配图片的特征索引
print(matches[0][0].trainIdx, matches[0][0].queryIdx)
print(matches[0][1].trainIdx, matches[0][1].queryIdx)
print(matches[1][0].trainIdx, matches[1][0].queryIdx)
print(matches[1][1].trainIdx, matches[1][1].queryIdx)
print(matches[2][0].trainIdx, matches[2][0].queryIdx)
print(matches[2][1].trainIdx, matches[2][1].queryIdx)

[<DMatch 0000019A05886EF0>, <DMatch 0000019A06566530>]
336 0
717 0
336 1
717 1
941 2
851 2


In [65]:
img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches[:3], None, flags=2)
cv_show(img3)

In [44]:
matches = list(np.array(matches).flatten())

img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:4], None, flags=2)
cv_show(img3)

## 全景拼接

In [94]:
import numpy as np
import cv2

class Stitcher(object):
    
    # 拼接函数
    def stitch(self, imgs, ratio=0.75, reprojThresh=4):
        """
        reprojThresh: RANSAC的重投影门限，即最小特征对数
        """
        imgB, imgA = imgs
        kpsA, featuresA = self.detectAndDescribe(imgA)
        kpsB, featuresB = self.detectAndDescribe(imgB)

        ret = self.matchKeyPoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)

        if ret is None: return None

        (matches, H, status) = ret
        
        # 将图片A进行变换
        result = cv2.warpPerspective(imgA, H, dsize=(imgA.shape[1]+imgB.shape[1], imgA.shape[0]))

        # 将图片B覆盖到A上
        result[0:imgB.shape[0], 0:imgB.shape[1]] = imgB

        # vis = self.drawMatches(imgA, imgB, kpsA, kpsB, matches, status)
        # 返回结果
        return result
        
    def cv_show(self,name,img):
        cv2.imshow(name, img)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 

    def detectAndDescribe(self, img):
        """
        得到图片的特征点和描述
        """
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        sift = cv2.xfeatures2d.SIFT_create()
        kps, features = sift.detectAndCompute(img, None)

        # ! 将结果转为 numpy 数组
        # pt 为特征点坐标
        kps = np.float32([kp.pt for kp in kps])
        # print(kps[:3])
        """
        [[  2.8513205 362.22076  ]
        [  5.0001354 229.0246   ]
        [  5.9197574 139.2678   ]]
        """
        return kps, features
    
    def matchKeyPoints(self, kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh):
        match = cv2.BFMatcher()
        matches_raw = match.knnMatch(featuresA, featuresB, 2)
        
        matches = []

        for m in matches_raw:
            if m[0].distance < ratio * m[1].distance:
                matches.append([m[0].queryIdx, m[0].trainIdx])
        
        # ptsA, ptsB 分别是 图A和图B的特征点坐标数组
        if len(matches) >= 4:
            ptsA =np.float32([kpsA[i] for i,_ in matches])
            ptsB =np.float32([kpsB[i] for _,i in matches])

            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

            return matches, H, status
        
        return None

    def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
        # 初始化可视化图片，将A、B图左右连接到一起
        (hA, wA) = imageA.shape[:2]
        (hB, wB) = imageB.shape[:2]
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imageA
        vis[0:hB, wA:] = imageB

        # 联合遍历，画出匹配对
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            # 当点对匹配成功时，画到可视化图上
            if s == 1:
                # 画出匹配对
                ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
                ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

        # 返回可视化结果
        return vis

In [105]:
import cv2
# 读取拼接图片
imageA = cv2.imread("01_Picture/demo_left.jpg")
imageB = cv2.imread("01_Picture/demo_right.jpg")
# cv_show(imageA)
# cv_show(imageB)

stitcher = Stitcher()
# vis: 匹配结果
result = stitcher.stitch([imageA, imageB])
cv2.imwrite("stitcher_pic.jpg", result)

True

In [102]:
# 拼接三张
import cv2
# 读取拼接图片
imageA = cv2.imread("01_Picture/1.jpg")
imageB = cv2.imread("01_Picture/2.jpg")
imageC = cv2.imread("01_Picture/3.jpg")
# cv_show(imageA)
# cv_show(imageB)

stitcher = Stitcher()
# vis: 匹配结果
imageAB = stitcher.stitch([imageA, imageB])
result = stitcher.stitch([imageAB, imageC])
cv2.imwrite("stitcher_pic.jpg", result)

True